In [109]:
import numpy as np
import scipy

def Kendall_Tau_Family(contingency_table, confidence_level = 0.95): #inspired by Desctools function
    number_of_rows, number_of_columns = contingency_table.shape
    concordant_table = np.zeros((number_of_rows, number_of_columns))
    disconcordant_table = np.zeros((number_of_rows, number_of_columns))
    for i in range(number_of_rows):
        for j in range(number_of_columns):
            concordant_table[i, j] = np.sum(contingency_table[:i, :j]) + np.sum(contingency_table[i+1:, j+1:])
            disconcordant_table[i, j] = np.sum(contingency_table[:i, j+1:]) + np.sum(contingency_table[i+1:, :j])
    Concordant_pairs, Disconcordant_pairs = np.sum(concordant_table * contingency_table) / 2, np.sum(disconcordant_table * contingency_table) / 2
    sample_size = np.sum(contingency_table)
    number_of_total_pairs = sample_size * (sample_size-1)/2  
    sum_of_the_rows = contingency_table.sum(axis=1)
    sum_of_the_cols = contingency_table.sum(axis=0)
    Number_of_ties_rows = np.sum(sum_of_the_rows * (sum_of_the_rows-1)/2)
    Number_of_ties_columns = np.sum(sum_of_the_cols * (sum_of_the_cols-1)/2)
    low_rc = (min(number_of_rows, number_of_columns)) # choose whethehr there are less row or coloumns

    Kendall_tau_a = (Concordant_pairs - Disconcordant_pairs) / number_of_total_pairs
    Kendall_tau_b = (Concordant_pairs - Disconcordant_pairs) / np.sqrt((number_of_total_pairs-Number_of_ties_rows) * (number_of_total_pairs-Number_of_ties_columns))
    Kendall_tau_c = 2 * (Concordant_pairs - Disconcordant_pairs) / (sample_size**2 *  ((low_rc - 1) / low_rc)) 

    # Standard Errors of tau
    Term1 = (concordant_table - disconcordant_table) * (contingency_table !=0)
    Term2 = np.sum(Term1[Term1 != 0]) / sample_size
    Standard_Error_Tau_a = np.sqrt(2 / (sample_size * (sample_size - 1)) * ((2 * (sample_size - 2)) / (sample_size * (sample_size - 1)**2) * np.sum((Term1 - Term2)**2) + 1 - Kendall_tau_a**2))
    Proportions_Tables = contingency_table / sample_size
    Proportions_Difference = (concordant_table - disconcordant_table) / sample_size
    sum_of_proportions_rows = np.sum(Proportions_Tables, axis=1)
    sum_of_proportions_columns = np.sum(Proportions_Tables, axis=0)
    proportions_matrix_row = (np.tile(sum_of_proportions_rows, (contingency_table.shape[1], 1)).T) * (2 * (Concordant_pairs - Disconcordant_pairs) / sample_size**2)
    proportions_matrix_columns = np.tile(sum_of_proportions_columns, (contingency_table.shape[0], 1)) * (2 * (Concordant_pairs - Disconcordant_pairs) / sample_size**2)
    delta1 = np.sqrt(1 - np.sum(sum_of_proportions_rows**2))
    delta2 = np.sqrt(1 - np.sum(sum_of_proportions_columns**2))
    Term3 = (2 * Proportions_Difference + proportions_matrix_columns) * delta2 * delta1 + (proportions_matrix_row * delta2) / delta1
    Standard_Error_Tau_b = np.sqrt(((np.sum(Proportions_Tables * Term3**2) - np.sum(Proportions_Tables * Term3)**2) / (delta1 * delta2)**4) / sample_size)
    Standard_Error_Tau_c = np.sqrt(4 * low_rc**2/((low_rc - 1)**2 * sample_size**4) * (np.sum(contingency_table * (concordant_table - disconcordant_table)**2) - 4 * (Concordant_pairs - Disconcordant_pairs)**2/sample_size))

    # Approximate Confidence Intervals Kendall's Tau
    zcrit = scipy.stats.t.ppf(1 - (1 - confidence_level) / 2, 100000)
    Confidence_Interval_Tau_a = (Kendall_tau_a - zcrit * Standard_Error_Tau_a,Kendall_tau_a + zcrit * Standard_Error_Tau_a)
    Confidence_Interval_Tau_b = (Kendall_tau_b - zcrit * Standard_Error_Tau_b,Kendall_tau_b + zcrit * Standard_Error_Tau_b)
    Confidence_Interval_Tau_c = (Kendall_tau_c - zcrit * Standard_Error_Tau_c,Kendall_tau_c + zcrit * Standard_Error_Tau_c)

    Z_Score_Tau_a = Kendall_tau_a / Standard_Error_Tau_a
    Z_Score_Tau_b = Kendall_tau_b / Standard_Error_Tau_b
    Z_Score_Tau_c = Kendall_tau_c / Standard_Error_Tau_c

    p_value_Tau_a = scipy.stats.t.sf((abs(Z_Score_Tau_a)), 100000) * 2
    p_value_Tau_b = scipy.stats.t.sf((abs(Z_Score_Tau_b)), 100000) * 2
    p_value_Tau_c = scipy.stats.t.sf((abs(Z_Score_Tau_c)), 100000) * 2

    results = {}

    results["Kendalls Tau-a:"]= Kendall_tau_a
    results["Kendalls Tau-a Standard Error"] = Standard_Error_Tau_a
    results["Kendalls Tau-a Z-score"] = Z_Score_Tau_a
    results["Kendalls Tau-a p-value"]= p_value_Tau_a
    results["Kendalls Tau-a Confidence Intervals"]= Confidence_Interval_Tau_a
    results["Kendalls Tau-b:"]= Kendall_tau_b
    results["Kendalls Tau-b Standard Error"] = Standard_Error_Tau_b
    results["Kendalls Tau-b Z-score"] = Z_Score_Tau_b
    results["Kendalls Tau-b p-value"]= p_value_Tau_b
    results["Kendalls Tau-b. Confidence Intervals"]= Confidence_Interval_Tau_b
    results["Kendalls Tau-c"]= Kendall_tau_c
    results["Kendalls Tau-c Standard Error"] = Standard_Error_Tau_c
    results["Kendalls Tau-c Z-score"] = Z_Score_Tau_c
    results["Kendalls Tau-c p-value"]= p_value_Tau_c
    results["Kendalls Tau-c Confidence Intervals"]= Confidence_Interval_Tau_c


    result_str = "\n".join([f"{key}: {value}" for key, value in results.items()])
    return result_str

# Your data
contingency_table = np.array([[2, 3], [4, 5], [7, 8]])
result = Kendall_Tau_Family(contingency_table)
print(result)


Kendalls Tau-a:: -0.024630541871921183
Kendalls Tau-a Standard Error: 0.07096158749883964
Kendalls Tau-a Z-score: -0.34709682717179824
Kendalls Tau-a p-value: 0.7285193074757812
Kendalls Tau-a Confidence Intervals: (-0.16371438107678848, 0.11445329733294612)
Kendalls Tau-b:: -0.043420842143242176
Kendalls Tau-b Standard Error: 0.17622674446405345
Kendalls Tau-b Z-score: -0.24639189854692636
Kendalls Tau-b p-value: 0.8053793827230222
Kendalls Tau-b. Confidence Intervals: (-0.38882309503162965, 0.30198141074514534)
Kendalls Tau-c: -0.04756242568370987
Kendalls Tau-c Standard Error: 0.19309448012256167
Kendalls Tau-c Z-score: -0.24631685822153418
Kendalls Tau-c p-value: 0.8054374664915745
Kendalls Tau-c Confidence Intervals: (-0.4260252331167167, 0.3309003817492969)
